# RAG System for Sport Articles

## Objectives:
1. Develop an RAG system using the semantic retriever from the previous assignment
2. Evaluate the performance of the RAG system

## Pipeline:
```
User Query → LLM Query Understanding → Optimized Query → FAISS (Chunk Retrieval) → Retrieved Chunks → Prompt Construction → LLM Response → Grounded Response
```

## Key Features:
- **Query Understanding**: LLM transforms natural language questions into search-optimized queries
- **Chunk-based Retrieval**: FAISS retrieves relevant **chunks** (500 chars with 100 overlap) instead of full articles
- **Semantic Search**: Multilingual E5 encoder for Indonesian sports articles
- **Grounded Generation**: Qwen2.5 generates responses based on retrieved context

## Step 0: Load Pre-trained Retriever

Load the embeddings, FAISS index, and **chunk metadata** from `assignment-non-rag.ipynb`.

**Important:** The FAISS index now points to **chunks** (sliding window segments), not full articles.
- `chunks_metadata.json`: Contains chunk content, article_id, chunk_id
- `article_embeddings.npy`: Embeddings for each chunk
- `faiss_index.bin`: FAISS index mapping to chunks

In [1]:
import json
import os
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer

# Paths
DATA_PATH = "scraping_result/detik_sport_articles_cleaned.json"
EMBEDDINGS_PATH = "embedding/article_embeddings.npy"
CHUNKS_PATH = "embedding/chunks_metadata.json"  # Chunk metadata from sliding window
INDEX_PATH = "faiss/faiss_index.bin"
MODEL_NAME = "intfloat/multilingual-e5-base"

# Verify files exist
required_files = [DATA_PATH, EMBEDDINGS_PATH, INDEX_PATH, CHUNKS_PATH]
for file_path in required_files:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"❌ {file_path} not found. Run assignment-non-rag.ipynb first!")

# Load original articles (for reference)
with open(DATA_PATH, "r", encoding="utf-8") as f:
    articles = json.load(f)
df = pd.DataFrame(articles)
df = df[df['content'].str.len() > 50].reset_index(drop=True)

# Load chunk metadata (THIS IS WHAT FAISS INDEX POINTS TO)
with open(CHUNKS_PATH, 'r', encoding='utf-8') as f:
    chunks_metadata = json.load(f)
df_chunks = pd.DataFrame(chunks_metadata)

# Load embeddings and FAISS index
embeddings = np.load(EMBEDDINGS_PATH)
index = faiss.read_index(INDEX_PATH)

# Load encoder model
print(f"Loading encoder model: {MODEL_NAME}")
encoder_model = SentenceTransformer(MODEL_NAME, device="cpu")

print(f"✅ Loaded {len(df)} original articles")
print(f"✅ Loaded {len(df_chunks)} chunks (from sliding window)")
print(f"✅ Loaded embeddings: {embeddings.shape}")
print(f"✅ Loaded FAISS index: {index.ntotal} vectors")
print(f"✅ Encoder model ready")

# Verify alignment
if len(df_chunks) != index.ntotal:
    print(f"⚠️ WARNING: Chunk count ({len(df_chunks)}) != FAISS vectors ({index.ntotal})")
    print("   Please re-run assignment-non-rag.ipynb to regenerate embeddings and index")
else:
    print(f"✅ Chunks and FAISS index are aligned")

Loading encoder model: intfloat/multilingual-e5-base
✅ Loaded 396 original articles
✅ Loaded 2325 chunks (from sliding window)
✅ Loaded embeddings: (2325, 768)
✅ Loaded FAISS index: 2325 vectors
✅ Encoder model ready
✅ Chunks and FAISS index are aligned
✅ Loaded 396 original articles
✅ Loaded 2325 chunks (from sliding window)
✅ Loaded embeddings: (2325, 768)
✅ Loaded FAISS index: 2325 vectors
✅ Encoder model ready
✅ Chunks and FAISS index are aligned


## Step 1: Select Generation Component

### Model Choice: **Qwen2.5:7b-instruct**

**Key reasons:**
1. **Better Indonesian understanding** - Our articles are in Indonesian
2. **Larger context window** - Can include more retrieved passages
3. **Better instruction following** - More likely to answer based on context only

In [2]:
import requests

# Ollama API configuration
OLLAMA_BASE_URL = "http://localhost:11434"
LLM_MODEL = "qwen2.5:7b-instruct"  # or "qwen2.5:7b" if instruct not available

def check_ollama_status():
    """Check if Ollama is running and model is available"""
    try:
        # Check if Ollama is running
        response = requests.get(f"{OLLAMA_BASE_URL}/api/tags")
        if response.status_code == 200:
            models = response.json().get('models', [])
            model_names = [m['name'] for m in models]
            print(f"✅ Ollama is running")
            print(f"📦 Available models: {model_names}")

            # Check if our model is available
            if any(LLM_MODEL in name for name in model_names):
                print(f"✅ Model '{LLM_MODEL}' is available")
                return True
            else:
                print(f"⚠️ Model '{LLM_MODEL}' not found. Please run: ollama pull {LLM_MODEL}")
                return False
        else:
            print("❌ Ollama API returned error")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to Ollama. Please start Ollama first: ollama serve")
        return False

# Check Ollama status
ollama_ready = check_ollama_status()

✅ Ollama is running
📦 Available models: ['qwen2.5:7b-instruct']
✅ Model 'qwen2.5:7b-instruct' is available


## Step 2: Build RAG Pipeline

Components:
1. **Retriever**: Reuse FAISS index from Part 1
2. **Prompt Constructor**: Combine query + retrieved passages
3. **Generator**: Ollama with Qwen2.5

### Query Understanding with LLM

Before sending the query to FAISS, we use the LLM to:
1. **Extract key entities** (names, events, dates)
2. **Reformulate** into a search-friendly query
3. **Remove noise** (question words, filler words)

**Flow:**
```
User Query → LLM Query Understanding → Optimized FAISS Query → Retrieval → Response Generation
```

In [3]:
def understand_query(user_query: str, verbose: bool = True) -> str:
    """
    Use LLM to transform user query into an optimized search query for FAISS.

    This extracts key entities and reformulates the query for better retrieval.
    """
    prompt = f"""Kamu adalah query optimizer untuk sistem pencarian artikel olahraga Indonesia.

    TUGAS: Ubah pertanyaan pengguna menjadi query pencarian yang optimal.

    ATURAN:
    1. Ekstrak entitas penting (nama pemain, tim, kompetisi, tanggal)
    2. Hilangkan kata tanya (siapa, apa, mengapa, bagaimana, kapan)
    3. Hilangkan kata penghubung yang tidak perlu
    4. Gunakan kata kunci yang kemungkinan ada di artikel berita
    5. Output HANYA query pencarian, tanpa penjelasan

    CONTOH:
    - Input: "Siapa yang keluar sebagai Juara Dunia MotoGP musim 2025?"
    - Output: juara dunia MotoGP 2025

    - Input: "Mengapa Rashford kesulitan di Manchester United?"
    - Output: kondisi Rashford Manchester United masalah performa

    - Input: "Rangkum pencapaian tim bulutangkis Indonesia di ajang Australia Open 2025"
    - Output: tim bulutangkis Indonesia Australia Open 2025 pencapaian hasil

    - Input: "Bagaimana performa Timnas Indonesia di Piala AFF?"
    - Output: Timnas Indonesia Piala AFF performa hasil

    PERTANYAAN PENGGUNA: {user_query}

    QUERY PENCARIAN:"""

    try:
        response = requests.post(
            f"{OLLAMA_BASE_URL}/api/generate",
            json={
                "model": LLM_MODEL,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": 0.1,  # Low temperature for consistent output
                    "num_predict": 50    # Short output expected
                }
            },
            timeout=60
        )

        if response.status_code == 200:
            optimized_query = response.json()['response'].strip()
            # Clean up - remove any extra explanation the LLM might add
            optimized_query = optimized_query.split('\n')[0].strip()

            if verbose:
                print(f"🔄 Query Understanding:")
                print(f"   Original: {user_query}")
                print(f"   Optimized: {optimized_query}")

            return optimized_query
        else:
            if verbose:
                print(f"⚠️ Query understanding failed, using original query")
            return user_query

    except Exception as e:
        if verbose:
            print(f"⚠️ Query understanding error: {e}, using original query")
        return user_query


In [4]:
def retrieve_passages(query: str, top_k: int = 3, use_query_understanding: bool = True, verbose: bool = True) -> tuple[list[dict], str]:
    """
    Retrieve relevant passages (chunks) using FAISS index.

    Args:
        query: User's original query
        top_k: Number of passages to retrieve
        use_query_understanding: Whether to use LLM to optimize query first
        verbose: Print debug information

    Returns:
        tuple: (passages, optimized_query)
    """
    # Step 1: Query Understanding (optional)
    if use_query_understanding:
        optimized_query = understand_query(query, verbose=verbose)
    else:
        optimized_query = query
        if verbose:
            print(f"🔍 Using original query: {query}")

    # Step 2: Encode query with E5 prefix
    query_with_prefix = f"query: {optimized_query}"
    query_embedding = encoder_model.encode(
        [query_with_prefix],
        normalize_embeddings=True,
        convert_to_numpy=True
    )

    # Step 3: Search FAISS index
    scores, indices = index.search(query_embedding.astype('float32'), top_k)

    # Step 4: Get passages from CHUNKS (not full articles!)
    passages = []
    for score, idx in zip(scores[0], indices[0]):
        chunk = df_chunks.iloc[idx]
        passages.append({
            'title': chunk['title'],
            'content': chunk['content'],  # This is chunk content, not full article
            'date': chunk['date'],
            'score': float(score),
            'article_id': chunk['article_id'],
            'chunk_id': chunk['chunk_id'],
            'chunk_info': f"Chunk {chunk['chunk_id']+1}/{chunk['total_chunks']}"
        })

    return passages, optimized_query


def construct_rag_prompt(query: str, passages: list[dict], max_content_len: int = 1000) -> str:
    """
    Construct a prompt that combines the query with retrieved passages (chunks).
    Instructs the LLM to answer based ONLY on the provided context.

    Args:
        query: User question (original, not optimized)
        passages: Retrieved chunks from FAISS
        max_content_len: Maximum characters per passage content
    """
    # Format passages - chunks are already shorter, so we can use more content
    context_parts = []
    for i, p in enumerate(passages, 1):
        content = p['content'][:max_content_len]
        context_parts.append(f"[{i}] {p['title']} ({p['chunk_info']})\n{content}")

    context = "\n\n".join(context_parts)

    prompt = f"""Jawab pertanyaan berdasarkan konteks berikut. Jika tidak ada informasi, katakan "Tidak tersedia".

KONTEKS:
{context}

PERTANYAAN: {query}

JAWABAN:"""

    return prompt


def construct_non_rag_prompt(query: str) -> str:
    """
    Construct a prompt WITHOUT retrieved passages.
    """
    prompt = f"""Jawab pertanyaan berikut dalam Bahasa Indonesia. PERTANYAAN: {query}"""

    return prompt


print("✅ RAG pipeline functions defined (with Chunk-based Retrieval)")

✅ RAG pipeline functions defined (with Chunk-based Retrieval)


In [5]:
import time

def generate_with_ollama(prompt: str, model: str = LLM_MODEL, timeout: int = 360) -> tuple[str, float]:
    """
    Generate response using Ollama API.

    Args:
        prompt: The prompt to send to the LLM
        model: Ollama model name
        timeout: Request timeout in seconds (default 180 = 3 minutes)

    Returns:
        tuple: (response_text, elapsed_time)
    """
    start_time = time.time()

    try:
        response = requests.post(
            f"{OLLAMA_BASE_URL}/api/generate",
            json={
                "model": model,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": 0.3,
                    "top_p": 0.9,
                    "num_predict": 256
                }
            },
            timeout=timeout
        )

        elapsed = time.time() - start_time

        if response.status_code == 200:
            return response.json()['response'], elapsed
        else:
            return f"Error: {response.status_code} - {response.text}", elapsed
    except requests.exceptions.Timeout:
        elapsed = time.time() - start_time
        return f"Error: Request timed out after {elapsed:.0f}s. Try a smaller model or shorter context.", elapsed
    except Exception as e:
        elapsed = time.time() - start_time
        return f"Error: {str(e)}", elapsed


def rag_query(query: str, top_k: int = 3, use_query_understanding: bool = True, verbose: bool = True) -> dict:
    """
    Complete RAG pipeline: Query Understanding → Retrieve → Construct Prompt → Generate

    Args:
        query: User's original query
        top_k: Number of passages to retrieve
        use_query_understanding: Whether to use LLM to optimize query for FAISS
        verbose: Print debug information
    """
    # Step 1: Retrieve relevant passages (with optional query understanding)
    passages, optimized_query = retrieve_passages(
        query,
        top_k=top_k,
        use_query_understanding=use_query_understanding,
        verbose=verbose
    )

    if verbose:
        print(f"📚 Retrieved {len(passages)} passages:")
        for i, p in enumerate(passages, 1):
            print(f"   {i}. [{p['score']:.4f}] {p['title']}")
        print()

    # Step 2: Construct RAG prompt (use ORIGINAL query for response generation)
    rag_prompt = construct_rag_prompt(query, passages)

    if verbose:
        print(f"📊 Prompt: {len(rag_prompt)} chars (~{len(rag_prompt)//4} tokens)")
        print("🤖 Generating RAG response...")

    # Step 3: Generate response
    rag_response, elapsed = generate_with_ollama(rag_prompt)

    if verbose:
        print(f"   ⏱️ Generated in {elapsed:.1f}s")

    return {
        'query': query,
        'optimized_query': optimized_query,
        'passages': passages,
        'rag_response': rag_response,
        'rag_prompt': rag_prompt,
        'elapsed': elapsed
    }


def non_rag_query(query: str, verbose: bool = True) -> dict:
    """
    Generate response WITHOUT retrieval (for comparison)
    """
    prompt = construct_non_rag_prompt(query)

    if verbose:
        print(f"🔍 Query: {query}")
        print("🤖 Generating non-RAG response (no context)...")

    response, elapsed = generate_with_ollama(prompt)

    if verbose:
        print(f"   ⏱️ Generated in {elapsed:.1f}s")

    return {
        'query': query,
        'non_rag_response': response,
        'elapsed': elapsed
    }


print("✅ Generation functions defined (with Query Understanding support)")

✅ Generation functions defined (with Query Understanding support)


## Step 3: Evaluation

Test the RAG system with different types of questions:
1. **Factual queries** - Specific facts that should be in the articles
2. **Conceptual understanding** - Questions requiring comprehension
3. **Synthesis questions** - Questions requiring combining information from multiple sources

In [6]:
# Define test questions for different categories
test_questions = {
    "factual": [
        "Siapa pelatih baru Timnas Indonesia?",
        "Berapa gol yang dicetak Ronaldo di Piala Dunia 2026?",
        "Kapan Barcelona membeli pemain baru terakhir kali?"
    ],
    "conceptual": [
        "Mengapa Manchester United mengalami masalah performa musim ini?",
        "Apa alasan Rashford kesulitan di Manchester United?",
        "Bagaimana kondisi klasemen Liga Inggris saat ini?"
    ],
    "synthesis": [
        "Bandingkan performa Ronaldo dan pemain bintang lainnya di kompetisi internasional",
        "Apa dampak pergantian pelatih terhadap performa tim-tim besar Eropa?",
        "Bagaimana situasi transfer pemain di liga-liga top Eropa?"
    ]
}

print("📋 Test Questions Defined:")
for category, questions in test_questions.items():
    print(f"\n{category.upper()}:")
    for i, q in enumerate(questions, 1):
        print(f"   {i}. {q}")

📋 Test Questions Defined:

FACTUAL:
   1. Siapa pelatih baru Timnas Indonesia?
   2. Berapa gol yang dicetak Ronaldo di Piala Dunia 2026?
   3. Kapan Barcelona membeli pemain baru terakhir kali?

CONCEPTUAL:
   1. Mengapa Manchester United mengalami masalah performa musim ini?
   2. Apa alasan Rashford kesulitan di Manchester United?
   3. Bagaimana kondisi klasemen Liga Inggris saat ini?

SYNTHESIS:
   1. Bandingkan performa Ronaldo dan pemain bintang lainnya di kompetisi internasional
   2. Apa dampak pergantian pelatih terhadap performa tim-tim besar Eropa?
   3. Bagaimana situasi transfer pemain di liga-liga top Eropa?


### Test 1: Factual Query

In [7]:
# DEBUG: See what's being sent to the LLM (UNTUK DEBUGGING AJA)
test_query = "Siapa yang keluar sebagai Juara Dunia MotoGP musim 2025?"
passages, optimized_query = retrieve_passages(test_query, top_k=3, use_query_understanding=True)
rag_prompt = construct_rag_prompt(test_query, passages)

🔄 Query Understanding:
   Original: Siapa yang keluar sebagai Juara Dunia MotoGP musim 2025?
   Optimized: juara dunia MotoGP 2025


In [8]:
# Ini buat debugging juga
print("📊 PROMPT STATISTICS:")
print(f"   - Original query: {test_query}")
print(f"   - Optimized query: {optimized_query}")
print(f"   - Total characters: {len(rag_prompt)}")
print(f"   - Estimated tokens: ~{len(rag_prompt) // 4} tokens")
print(f"   - Number of passages: {len(passages)}")
print()
print("📝 FULL PROMPT SENT TO LLM:")
print("=" * 80)
print(rag_prompt)
print("=" * 80)

📊 PROMPT STATISTICS:
   - Original query: Siapa yang keluar sebagai Juara Dunia MotoGP musim 2025?
   - Optimized query: juara dunia MotoGP 2025
   - Total characters: 1833
   - Estimated tokens: ~458 tokens
   - Number of passages: 3

📝 FULL PROMPT SENT TO LLM:
Jawab pertanyaan berdasarkan konteks berikut. Jika tidak ada informasi, katakan "Tidak tersedia".

KONTEKS:
[1] Jadwal MotoGP Valencia 2025: Bakal Ada 7 Pemenang Berbeda Beruntun? (Chunk 2/6)
oGP 2025, race di Ricardo Tormo, Valencia, pada akhir pekan ini bakal menjadi balapan penutup musim. Saat ini Marc Marquez sudah dipastikan menjadi juara dunia MotoGP 2025 . Adik kandungnya, Alex Marquez, juga sudah mengunci posisi kedua di klasemen MotoGP 2025 . Secara matematis, posisi itu menjadi rebutan Marco Bezzecchi dan Francesco 'Pecco' Bagnaia . Saat ini Bezz berada di posisi yang lebih baik ketimbang Pecco untuk bisa memastikan posisi ketiga di akhir musim.

[2] Klasemen Akhir MotoGP 2025: Marc Marquez No 1, Disusul Alex dan Bezz

In [9]:
# Factual Query Test
factual_query = "Apa julukan yang dimiliki oleh klub sepakbola Ajax Amsterdam?"

# RAG Response
factual_rag_result = rag_query(factual_query, top_k=3)

# Non-RAG Response
factual_non_rag_result = non_rag_query(factual_query)

🔄 Query Understanding:
   Original: Apa julukan yang dimiliki oleh klub sepakbola Ajax Amsterdam?
   Optimized: juulan ajax amsterdam klub sepakbola julukan
📚 Retrieved 3 passages:
   1. [0.8892] Profil Ajax: Filosofi Total Football dan Pabrik Bintang Dunia
   2. [0.8693] Profil Ajax: Filosofi Total Football dan Pabrik Bintang Dunia
   3. [0.8646] Profil Ajax: Filosofi Total Football dan Pabrik Bintang Dunia

📊 Prompt: 1685 chars (~421 tokens)
🤖 Generating RAG response...
   ⏱️ Generated in 54.0s
🔍 Query: Apa julukan yang dimiliki oleh klub sepakbola Ajax Amsterdam?
🤖 Generating non-RAG response (no context)...
   ⏱️ Generated in 54.0s
🔍 Query: Apa julukan yang dimiliki oleh klub sepakbola Ajax Amsterdam?
🤖 Generating non-RAG response (no context)...
   ⏱️ Generated in 23.0s
   ⏱️ Generated in 23.0s


In [10]:
print("=" * 80)
print("FACTUAL QUERY TEST")
print("=" * 80)

print("\n📗 RAG RESPONSE:")
print("-" * 40)
print(factual_rag_result['rag_response'])

print("\n" + "=" * 80)

print("\n📕 NON-RAG RESPONSE:")
print("-" * 40)
print(factual_non_rag_result['non_rag_response'])

FACTUAL QUERY TEST

📗 RAG RESPONSE:
----------------------------------------
Julukan yang dimiliki oleh klub sepakbola Ajax Amsterdam antara lain de Joden, Lucky Ajax, the Lancers, dan de Godenzonen.


📕 NON-RAG RESPONSE:
----------------------------------------
Klub sepakbola Ajax Amsterdam memiliki beberapa julukan, namun julukan terkenal dan paling umum adalah "Tim Raksasa" atau dalam bahasa Indonesia disebut "Tim Orang Besar". Julukan ini berasal dari bahasa Belanda yang berarti "Giant Club", di mana "giant" berarti raksasa.


### Test 2: Conceptual Understanding

In [11]:

# Conceptual Query Test
conceptual_query = "Mengapa Rashford kesulitan di Manchester United?"

# RAG Response
conceptual_rag_result = rag_query(conceptual_query, top_k=3)

# Non-RAG Response
conceptual_non_rag_result = non_rag_query(conceptual_query)

🔄 Query Understanding:
   Original: Mengapa Rashford kesulitan di Manchester United?
   Optimized: kondisi Rashford Manchester United masalah performa
📚 Retrieved 3 passages:
   1. [0.8790] Deco: Rashford Menderita di MU
   2. [0.8707] Deco: Rashford Menderita di MU
   3. [0.8625] Deco: Rashford Menderita di MU

📊 Prompt: 1608 chars (~402 tokens)
🤖 Generating RAG response...
   ⏱️ Generated in 77.8s
🔍 Query: Mengapa Rashford kesulitan di Manchester United?
🤖 Generating non-RAG response (no context)...
   ⏱️ Generated in 77.8s
🔍 Query: Mengapa Rashford kesulitan di Manchester United?
🤖 Generating non-RAG response (no context)...
   ⏱️ Generated in 55.1s
   ⏱️ Generated in 55.1s


In [12]:
print("=" * 80)
print("CONCEPTUAL UNDERSTANDING TEST")
print("=" * 80)

print("\n📗 RAG RESPONSE:")
print("-" * 40)
print(conceptual_rag_result['rag_response'])

print("\n" + "=" * 80)

print("\n📕 NON-RAG RESPONSE:")
print("-" * 40)
print(conceptual_non_rag_result['non_rag_response'])


CONCEPTUAL UNDERSTANDING TEST

📗 RAG RESPONSE:
----------------------------------------
Rashford kesulitan di Manchester United karena mengalami perubahan generasi di tim tersebut. Menurut Deco, ini tidak mudah bagi seorang pemain penting yang memiliki banyak tanggung jawab. Selama lima tahun terakhir, United kesulitan membangun kembali tim agar lebih kuat.


📕 NON-RAG RESPONSE:
----------------------------------------
Rashford mengalami beberapa tantangan di Manchester United yang dapat mempengaruhi performa dan posisinya sebagai pemain utama. Berikut beberapa alasan mengapa Rashford kesulitan di Manchester United:

1. Persaingan Ketat: Manchester United memiliki banyak pemain berkualitas di posisi serang, termasuk Marcus Rashford sendiri. Hal ini membuat persaingan untuk tempat bermain menjadi sangat ketat.

2. Kekuatan Mental: Beberapa laporan menyebutkan bahwa Rashford mengalami masalah kepercayaan diri dan tekanan mental yang cukup tinggi, terutama setelah beberapa musim tanpa gel

### Test 3: Synthesis Question

In [13]:
# Synthesis Query Test
synthesis_query = "Rangkum pencapaian tim bulutangkis Indonesia di ajang Australia Open 2025."

# RAG Response
synthesis_rag_result = rag_query(synthesis_query, top_k=5)  # More passages for synthesis

# Non-RAG Response
synthesis_non_rag_result = non_rag_query(synthesis_query)


🔄 Query Understanding:
   Original: Rangkum pencapaian tim bulutangkis Indonesia di ajang Australia Open 2025.
   Optimized: tim bulutangkis Indonesia Australia Open 2025 pencapaian hasil
📚 Retrieved 5 passages:
   1. [0.8906] Hasil Lengkap Australia Open 2025: RI Raih 2 Gelar Juara!
   2. [0.8847] PBSI-BNI Beri Dukungan Penuh untuk Atlet RI di Final Australia Open 2025
   3. [0.8728] PBSI-BNI Beri Dukungan Penuh untuk Atlet RI di Final Australia Open 2025
   4. [0.8682] Australia Open 2025: Hasil Pebulutangkis RI Hari Ini, Jojo Belum Main
   5. [0.8672] Australia Open 2025: Hasil Pebulutangkis RI Hari Ini, Jojo Belum Main

📊 Prompt: 2820 chars (~705 tokens)
🤖 Generating RAG response...
   ⏱️ Generated in 172.9s
🔍 Query: Rangkum pencapaian tim bulutangkis Indonesia di ajang Australia Open 2025.
🤖 Generating non-RAG response (no context)...
   ⏱️ Generated in 172.9s
🔍 Query: Rangkum pencapaian tim bulutangkis Indonesia di ajang Australia Open 2025.
🤖 Generating non-RAG response (no cont

In [14]:

print("=" * 80)
print("SYNTHESIS QUESTION TEST")
print("=" * 80)

print("\n📗 RAG RESPONSE:")
print("-" * 40)
print(synthesis_rag_result['rag_response'])

print("\n" + "=" * 80)

print("\n📕 NON-RAG RESPONSE:")
print("-" * 40)
print(synthesis_non_rag_result['non_rag_response'])

SYNTHESIS QUESTION TEST

📗 RAG RESPONSE:
----------------------------------------
Tim bulutangkis Indonesia mencapai prestasi luar biasa di ajang BWF Super 500 Australia Open 2025. Dua partai final nasional terjadi, yaitu dalam nomor ganda putri dan ganda putra. Meskipun informasi spesifik tentang pemenang final tidak disebutkan, dapat diketahui bahwa Indonesia meraih dua gelar juara dari rangkaian pertandingan final tersebut.

Selain pencapaian di final, beberapa atlet Indonesia juga berhasil lolos ke babak 32 besar. Hasil yang dicapai antara lain:

- Sabar Karyaman Gutama/Moh. Reza Pahlevi Isfahani menang atas Hiroki Midorikawa/Kyohei Yamashita
- Alwi Farhan memenangkan pertandingan dengan withdraw lawannya 
- Putri Kusuma Wardani mengalahkan Sung Shuo Yun dengan skor 21-12, 21-8
- Adnan Maulana/Indah Cahya Sari Jamil menang atas Edward Lau/Shaunna Li dengan skor 21-14,


📕 NON-RAG RESPONSE:
----------------------------------------
Maaf, untuk memberikan rangkuman tentang pencapaian 

### Manual Evaluation

Use the cell below to manually score the responses. Fill in scores 1-5 for each criterion.

In [ ]:
# Manual Evaluation Scores (Fill in after reviewing responses above)
# Score range: 1 (Poor) to 5 (Excellent)

manual_scores = {
    "RAG": {
        "Factual": {
            "groundedness": 5,      # Is answer based on retrieved passages?
            "factual_accuracy": 4,  # Are facts correct?
            "no_hallucination": 3,  # Does it avoid making things up?
            "relevance": 5,         # Does it answer the question?
            "completeness": 5       # Is the answer complete?
        },
        "Conceptual": {
            "groundedness": 5,
            "factual_accuracy": 5,
            "no_hallucination": 3,
            "relevance": 4,
            "completeness": 5
        },
        "Synthesis": {
            "groundedness": 5,
            "factual_accuracy": 5,
            "no_hallucination": 5,
            "relevance": 5,
            "completeness": 5
        }
    },
    "Non-RAG": {
        "Factual": {
            "groundedness": 5,
            "factual_accuracy": 5,
            "no_hallucination": 5,
            "relevance": 5,
            "completeness": 5
        },
        "Conceptual": {
            "groundedness": 5,
            "factual_accuracy": 2,
            "no_hallucination": 3,
            "relevance": 5,
            "completeness": 4
        },
        "Synthesis": {
            "groundedness": 5,
            "factual_accuracy": 3,
            "no_hallucination": 4,
            "relevance": 4,
            "completeness": 5
        }
    }
}

print("📝 Manual Evaluation Template")
print("=" * 60)
print("\nInstructions:")
print("1. Review the RAG and Non-RAG responses from the tests above")
print("2. Fill in scores (1-5) in the manual_scores dictionary")
print("3. Run the next cell to see the summary")
print("\nScoring Guide:")
print("  1 = Poor       - Completely wrong or irrelevant")
print("  2 = Below Avg  - Major issues")
print("  3 = Average    - Acceptable but with issues")
print("  4 = Good       - Mostly correct with minor issues")
print("  5 = Excellent  - Fully correct and complete")

In [ ]:
def calculate_evaluation_summary(scores: dict) -> pd.DataFrame:
    """
    Calculate summary statistics from manual evaluation scores.
    """
    criteria = ["groundedness", "factual_accuracy", "no_hallucination", "relevance", "completeness"]

    summary = []
    for system in ["RAG", "Non-RAG"]:
        system_scores = scores[system]

        # Calculate averages across all queries
        avg_scores = {c: [] for c in criteria}

        for query_scores in system_scores.values():
            for criterion, score in query_scores.items():
                if score > 0:  # Only count filled scores
                    avg_scores[criterion].append(score)

        # Calculate final averages
        final_scores = {}
        for criterion, scores_list in avg_scores.items():
            final_scores[criterion] = sum(scores_list) / len(scores_list) if scores_list else 0

        overall = sum(final_scores.values()) / len(final_scores) if any(final_scores.values()) else 0

        summary.append({
            "System": system,
            "Groundedness": final_scores["groundedness"],
            "Factual Acc.": final_scores["factual_accuracy"],
            "No Halluc.": final_scores["no_hallucination"],
            "Relevance": final_scores["relevance"],
            "Completeness": final_scores["completeness"],
            "Overall": overall
        })

    return pd.DataFrame(summary)


# Calculate and display summary
print("📊 Evaluation Summary")
print("=" * 80)

# Check if scores have been filled
has_scores = any(
    score > 0
    for system in manual_scores.values()
    for query in system.values()
    for score in query.values()
)

if not has_scores:
    print("⚠️ No scores filled yet. Please fill in manual_scores above and re-run this cell.")
else:
    summary_df = calculate_evaluation_summary(manual_scores)
    print(summary_df.to_string(index=False))

    # Calculate winner for each criterion
    print("\n" + "-" * 80)
    print("🏆 Winner by Criterion:")
    for col in summary_df.columns[1:]:
        rag_score = summary_df.loc[summary_df['System'] == 'RAG', col].values[0]
        non_rag_score = summary_df.loc[summary_df['System'] == 'Non-RAG', col].values[0]
        if rag_score > non_rag_score:
            print(f"   {col}: RAG wins ({rag_score:.2f} vs {non_rag_score:.2f})")
        elif non_rag_score > rag_score:
            print(f"   {col}: Non-RAG wins ({non_rag_score:.2f} vs {rag_score:.2f})")
        else:
            print(f"   {col}: Tie ({rag_score:.2f})")

## RAG System Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                     RAG SYSTEM PIPELINE                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   User Query                                                    │
│       │                                                         │
│       ▼                                                         │
│   ┌─────────────────────────────────────┐                       │
│   │  QUERY UNDERSTANDING (Ollama)       │                       │
│   │  - Extract entities                 │                       │
│   │  - Optimize for search              │                       │
│   └─────────────────────────────────────┘                       │
│       │                                                         │
│       ▼                                                         │
│   ┌─────────────────────────────────────┐                       │
│   │  RETRIEVER (from assignment-non-rag)│                       │
│   │  - Multilingual E5 Encoder          │                       │
│   │  - FAISS Vector Index               │                       │
│   │  - Returns CHUNKS (not full docs)   │  ← Sliding Window     │
│   └─────────────────────────────────────┘    500 chars, 100 overlap
│       │                                                         │
│       ▼                                                         │
│   ┌─────────────────────────────────────┐                       │
│   │  PROMPT CONSTRUCTOR                 │                       │
│   │  - Combine Query + Chunks           │                       │
│   │  - Add Grounding Instructions       │                       │
│   └─────────────────────────────────────┘                       │
│       │                                                         │
│       ▼                                                         │
│   ┌─────────────────────────────────────┐                       │
│   │  GENERATOR (Ollama)                 │                       │
│   │  - Qwen2.5-7B-Instruct              │                       │
│   │  - Grounded Response Generation     │                       │
│   └─────────────────────────────────────┘                       │
│       │                                                         │
│       ▼                                                         │
│   Grounded Response                                             │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```